In [6]:
"""
Data Analysis & ML Preparation
================================================
This Code performs:
1. Detailed Statistical Analysis
2. Feature and class separability (PCA, t-SNE, UMAP)
3. Proper ML Data Structure Preparation
"""

import pandas as pd
import numpy as np
import json
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif, f_classif, SelectKBest
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Try to import UMAP
try:
    import umap
    UMAP_AVAILABLE = True
except ImportError:
    UMAP_AVAILABLE = False
    print("Warning: UMAP not installed. Run: pip install umap-learn")


# The input directory and final data directory or output directory
DATA_DIR = "./output"
INPUT_FILE = "cleaned_measurements.csv"
SUBSTANCES = ['Acetone', 'Redidlo', 'Vinegar', 'Softasept', 'Savo']

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

#Let us use Helper functions to Convert nested JSON sensor data into a flat dictionary for tabular analysis
def calculate_class_separability(X, y):
    """Calculate Fisher's Linear Discriminant ratio for class separability""" #to identify how well the class separtaed
    classes = np.unique(y)
    n_features = X.shape[1]
    
    # Overall mean
    overall_mean = np.mean(X, axis=0)
    
    # Between-class scatter
    Sb = np.zeros((n_features, n_features))
    # Within-class scatter
    Sw = np.zeros((n_features, n_features))
    
    for c in classes:
        X_c = X[y == c]
        mean_c = np.mean(X_c, axis=0)
        n_c = X_c.shape[0]
        
        # Between-class
        mean_diff = (mean_c - overall_mean).reshape(-1, 1)
        Sb += n_c * np.dot(mean_diff, mean_diff.T)
        
        # Within-class
        for x in X_c:
            diff = (x - mean_c).reshape(-1, 1)
            Sw += np.dot(diff, diff.T)
    
    # Fisher criterion (trace ratio)
    try:
        Sw_inv = np.linalg.pinv(Sw)
        fisher_ratio = np.trace(np.dot(Sw_inv, Sb))
    except:
        fisher_ratio = np.nan
    
    return fisher_ratio

def main():
    print("=" * 70)
    print("DATA ANALYSIS & ML PREPARATION")
    print("=" * 70)
    
    # Setup paths
    data_path = Path(DATA_DIR)
    input_file = data_path / INPUT_FILE
    
    if not input_file.exists():
        print(f"\n❌ ERROR: Input file not found: {input_file}")
        return None

    print("Step 1: Loading Data")

    
    df = pd.read_csv(input_file)
    print(f"\n✓ Loaded {len(df)} rows × {len(df.columns)} columns")
    
    # Identify columns
    meta_cols = ['id', 'box_id', 'ts', 'measurement_id', 'substance', 
                 'measurement_type', 'measurement_num', 'source_file', 'created_at']
    available_meta = [c for c in meta_cols if c in df.columns]
    numeric_cols = [c for c in df.columns if c not in available_meta]
    
    # Convert to numeric
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    print(f"✓ Metadata columns: {len(available_meta)}")
    print(f"✓ Numeric feature columns: {len(numeric_cols)}")
    
    # --- VERIFICATION ---
    print("\n>>> STEP 1 VERIFICATION:")
    print(f"    ✓ Data shape: {df.shape}")
    print(f"    ✓ Substances: {df['substance'].unique().tolist()}")
    print(f"    ✓ Measurement types: {df['measurement_type'].unique().tolist()}")
    
    print("Step 2: Detailed Statistical Analysis")
    # 2.1 Basic Statistics
    print("\n 2.1 Basic Statistics ")
    
    basic_stats = {
        'Total Samples': len(df),
        'Total Features': len(numeric_cols),
        'Substances': len(df['substance'].unique()),
        'Missing Values (%)': (df[numeric_cols].isnull().sum().sum() / (len(df) * len(numeric_cols)) * 100),
    }
    
    for key, value in basic_stats.items():
        print(f"  {key}: {value:.2f}" if isinstance(value, float) else f"  {key}: {value}")
    

    print("\n 2.2 Per-Substance Statistics ")
    substance_stats = []
    for substance in df['substance'].unique():
        subset = df[df['substance'] == substance]
        air_subset = subset[subset['measurement_type'] == 'air']
        subst_subset = subset[subset['measurement_type'] != 'air']
        
        stats_dict = {
            'Substance': substance,
            'Total_Samples': len(subset),
            'Air_Samples': len(air_subset),
            'Substance_Samples': len(subst_subset),
            'Air_Ratio': len(air_subset) / len(subset) * 100 if len(subset) > 0 else 0,
        }
        
        # Key feature statistics (for gas resistance)
        gas_res_cols = [c for c in numeric_cols if 'gas_res' in c]
        if gas_res_cols:
            gas_res_mean = subst_subset[gas_res_cols].mean().mean()
            gas_res_std = subst_subset[gas_res_cols].std().mean()
            stats_dict['Mean_Gas_Res'] = gas_res_mean
            stats_dict['Std_Gas_Res'] = gas_res_std
            stats_dict['CV_Gas_Res'] = (gas_res_std / gas_res_mean * 100) if gas_res_mean > 0 else 0
        
        substance_stats.append(stats_dict)
    
    substance_stats_df = pd.DataFrame(substance_stats)
    print(substance_stats_df.to_string(index=False))
    substance_stats_df.to_csv(data_path / 'detailed_substance_statistics.csv', index=False)
    
    # 2.3 Feature Statistics
    print("\n 2.3 Feature Statistics ")
    
    feature_stats = []
    for col in numeric_cols:
        col_data = df[col].dropna()
        if len(col_data) > 0:
            stats_dict = {
                'Feature': col,
                'Count': len(col_data),
                'Missing_Pct': (df[col].isnull().sum() / len(df) * 100),
                'Mean': col_data.mean(),
                'Std': col_data.std(),
                'Min': col_data.min(),
                'Q25': col_data.quantile(0.25),
                'Median': col_data.median(),
                'Q75': col_data.quantile(0.75),
                'Max': col_data.max(),
                'Skewness': stats.skew(col_data),
                'Kurtosis': stats.kurtosis(col_data),
            }
            feature_stats.append(stats_dict)
    
    feature_stats_df = pd.DataFrame(feature_stats)
    feature_stats_df.to_csv(data_path / 'detailed_feature_statistics.csv', index=False)
    print(f"✓ Saved detailed feature statistics ({len(feature_stats)} features)")
    
    print("\n 2.4 ANOVA Tests for Feature Significance ")
    # Filter to substance-only measurements
    df_subst = df[df['measurement_type'] != 'air'].copy()
    
    anova_results = []
    for col in numeric_cols:
        col_data = df_subst[col].dropna()
        if len(col_data) > 100:  # Minimum samples
            groups = [df_subst[df_subst['substance'] == s][col].dropna() for s in df_subst['substance'].unique()]
            groups = [g for g in groups if len(g) > 10]
            
            if len(groups) >= 2:
                try:
                    f_stat, p_value = stats.f_oneway(*groups)
                    anova_results.append({
                        'Feature': col,
                        'F_Statistic': f_stat,
                        'P_Value': p_value,
                        'Significant': p_value < 0.05
                    })
                except:
                    pass
    
    anova_df = pd.DataFrame(anova_results).sort_values('F_Statistic', ascending=False)
    anova_df.to_csv(data_path / 'anova_results.csv', index=False)
    
    print(f"\nTop 10 Most Discriminative Features (ANOVA):")
    print(anova_df.head(10).to_string(index=False))
    
    print("\n>>>PRINT VERIFICATION:")
    print(f"    ✓ Substance statistics saved: detailed_substance_statistics.csv")
    print(f"    ✓ Feature statistics saved: detailed_feature_statistics.csv")
    print(f"    ✓ ANOVA results saved: anova_results.csv")
    print(f"    ✓ Significant features (p<0.05): {anova_df['Significant'].sum()}/{len(anova_df)}")


    print("STEP 3: Feature Importance & Analysis") 
    # Prepare data for feature importance
    df_analysis = df_subst.copy()
    X_raw = df_analysis[numeric_cols].copy()
    
    # Impute missing values
    imputer = SimpleImputer(strategy='median')
    X_imputed = pd.DataFrame(imputer.fit_transform(X_raw), columns=numeric_cols, index=X_raw.index)
    
    # Encode labels
    le = LabelEncoder()
    y = le.fit_transform(df_analysis['substance'])
    
    # 3.1 Mutual Information
    print("\n 3.1 Mutual Information Scores ")
    
    mi_scores = mutual_info_classif(X_imputed, y, random_state=42)
    mi_df = pd.DataFrame({
        'Feature': numeric_cols,
        'MI_Score': mi_scores
    }).sort_values('MI_Score', ascending=False)
    mi_df.to_csv(data_path / 'mutual_information_scores.csv', index=False)
    
    print("Top 10 Features by Mutual Information:")
    print(mi_df.head(10).to_string(index=False))
    
    # 3.2 F-Score (ANOVA F-value)
    print("\n 3.2 F-Scores ")
    
    f_scores, f_pvalues = f_classif(X_imputed, y)
    f_df = pd.DataFrame({
        'Feature': numeric_cols,
        'F_Score': f_scores,
        'P_Value': f_pvalues
    }).sort_values('F_Score', ascending=False)
    f_df.to_csv(data_path / 'f_scores.csv', index=False)
    
    print("Top 10 Features by F-Score:")
    print(f_df.head(10).to_string(index=False))
    
    # 3.3 Combined Feature Ranking
    print("\n 3.3 Combined Feature Ranking ")
    
    # Normalize scores
    mi_df['MI_Rank'] = mi_df['MI_Score'].rank(ascending=False)
    f_df['F_Rank'] = f_df['F_Score'].rank(ascending=False)
    
    combined_df = mi_df.merge(f_df[['Feature', 'F_Score', 'F_Rank']], on='Feature')
    combined_df['Avg_Rank'] = (combined_df['MI_Rank'] + combined_df['F_Rank']) / 2
    combined_df = combined_df.sort_values('Avg_Rank')
    combined_df.to_csv(data_path / 'combined_feature_ranking.csv', index=False)
    
    print("Top 15 Features (Combined Ranking):")
    print(combined_df.head(15)[['Feature', 'MI_Score', 'F_Score', 'Avg_Rank']].to_string(index=False))
    
    # VERIFICATION 
    print("\n>>> PRINT VERIFICATION:")
    print(f"    ✓ Mutual Information scores saved")
    print(f"    ✓ F-Scores saved")
    print(f"    ✓ Combined ranking saved")
    print(f"    ✓ Top feature: {combined_df.iloc[0]['Feature']}")
    
    print("STEP 4: Dimensionality Reduction & Class Separability")
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    
    # Get substance labels for coloring
    substances = df_analysis['substance'].values
    
    # Color palette
    colors = {'Acetone': '#E91E63', 'Redidlo': '#FFC107', 'Vinegar': '#4CAF50', 
              'Softasept': '#00BCD4', 'Savo': '#3F51B5'}
    color_list = [colors.get(s, '#000000') for s in substances]
    
    # 4.1 PCA Analysis
    print("\n 4.1 PCA Analysis ")
    
    pca = PCA(n_components=min(10, X_scaled.shape[1]))
    X_pca = pca.fit_transform(X_scaled)
    
    # Explained variance
    explained_var = pca.explained_variance_ratio_
    cumulative_var = np.cumsum(explained_var)
    
    print(f"Explained Variance by Component:")
    for i in range(min(5, len(explained_var))):
        print(f"  PC{i+1}: {explained_var[i]*100:.2f}% (Cumulative: {cumulative_var[i]*100:.2f}%)")
    
    # PCA Plot
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Scree plot
    axes[0].bar(range(1, len(explained_var)+1), explained_var * 100, alpha=0.7, label='Individual')
    axes[0].plot(range(1, len(explained_var)+1), cumulative_var * 100, 'ro-', label='Cumulative')
    axes[0].axhline(y=95, color='g', linestyle='--', label='95% threshold')
    axes[0].set_xlabel('Principal Component')
    axes[0].set_ylabel('Explained Variance (%)')
    axes[0].set_title('PCA Scree Plot', fontweight='bold')
    axes[0].legend()
    
    # PC1 vs PC2
    for substance in SUBSTANCES:
        mask = substances == substance
        axes[1].scatter(X_pca[mask, 0], X_pca[mask, 1], c=colors[substance], 
                       label=substance, alpha=0.6, s=30)
    axes[1].set_xlabel(f'PC1 ({explained_var[0]*100:.1f}%)')
    axes[1].set_ylabel(f'PC2 ({explained_var[1]*100:.1f}%)')
    axes[1].set_title('PCA: PC1 vs PC2', fontweight='bold')
    axes[1].legend()
    
    # PC1 vs PC3
    for substance in SUBSTANCES:
        mask = substances == substance
        axes[2].scatter(X_pca[mask, 0], X_pca[mask, 2], c=colors[substance], 
                       label=substance, alpha=0.6, s=30)
    axes[2].set_xlabel(f'PC1 ({explained_var[0]*100:.1f}%)')
    axes[2].set_ylabel(f'PC3 ({explained_var[2]*100:.1f}%)')
    axes[2].set_title('PCA: PC1 vs PC3', fontweight='bold')
    axes[2].legend()
    
    plt.tight_layout()
    plt.savefig(data_path / '09_pca_analysis.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("✓ Saved: 09_pca_analysis.png")
    
    # PCA loadings
    loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(pca.n_components_)],
                           index=numeric_cols)
    loadings.to_csv(data_path / 'pca_loadings.csv')
    
    # 4.2 t-SNE Analysis
    print("\n 4.2 t-SNE Analysis ")
    
    # Use PCA-reduced data for t-SNE (faster and often better)
    n_components_tsne = min(50, X_pca.shape[1])
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    perplexities = [5, 30, 50]
    for idx, perp in enumerate(perplexities):
        print(f"  Running t-SNE with perplexity={perp}...")
        tsne = TSNE(n_components=2, perplexity=perp, random_state=42, max_iter=1000)
        X_tsne = tsne.fit_transform(X_pca[:, :n_components_tsne])
        
        for substance in SUBSTANCES:
            mask = substances == substance
            axes[idx].scatter(X_tsne[mask, 0], X_tsne[mask, 1], c=colors[substance], 
                            label=substance, alpha=0.6, s=30)
        axes[idx].set_xlabel('t-SNE 1')
        axes[idx].set_ylabel('t-SNE 2')
        axes[idx].set_title(f't-SNE (perplexity={perp})', fontweight='bold')
        axes[idx].legend()
    
    plt.tight_layout()
    plt.savefig(data_path / '10_tsne_analysis.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("✓ Saved: 10_tsne_analysis.png")
    
    # 4.3 UMAP Analysis
    if UMAP_AVAILABLE:
        print("\n  4.3 UMAP Analysis  ")
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        n_neighbors_list = [5, 15, 50]
        for idx, n_neighbors in enumerate(n_neighbors_list):
            print(f"  Running UMAP with n_neighbors={n_neighbors}...")
            reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=0.1, random_state=42)
            X_umap = reducer.fit_transform(X_scaled)
            
            for substance in SUBSTANCES:
                mask = substances == substance
                axes[idx].scatter(X_umap[mask, 0], X_umap[mask, 1], c=colors[substance], 
                                label=substance, alpha=0.6, s=30)
            axes[idx].set_xlabel('UMAP 1')
            axes[idx].set_ylabel('UMAP 2')
            axes[idx].set_title(f'UMAP (n_neighbors={n_neighbors})', fontweight='bold')
            axes[idx].legend()
        
        plt.tight_layout()
        plt.savefig(data_path / '11_umap_analysis.png', dpi=150, bbox_inches='tight')
        plt.close()
        print("✓ Saved: 11_umap_analysis.png")
    else:
        print("\n 4.3 UMAP Analysis (SKIPPED - not available) ")
    
    # 4.4 Class Separability Metrics
    print("\n 4.4 Class Separability Metrics ")
    
    # Silhouette-like metric per class
    from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
    
    separability_metrics = {
        'Silhouette_Score': silhouette_score(X_scaled, y),
        'Calinski_Harabasz_Index': calinski_harabasz_score(X_scaled, y),
        'Davies_Bouldin_Index': davies_bouldin_score(X_scaled, y),
        'Fisher_Criterion': calculate_class_separability(X_scaled, y)
    }
    
    print("Class Separability Metrics:")
    for metric, value in separability_metrics.items():
        print(f"  {metric}: {value:.4f}")
    
    # Save metrics
    with open(data_path / 'class_separability_metrics.json', 'w') as f:
        json.dump({k: float(v) for k, v in separability_metrics.items()}, f, indent=2)
    
    # 4.5 Combined Visualization
    print("\n 4.5 Combined Visualization ")
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 14))
    
    # PCA
    for substance in SUBSTANCES:
        mask = substances == substance
        axes[0, 0].scatter(X_pca[mask, 0], X_pca[mask, 1], c=colors[substance], 
                          label=substance, alpha=0.6, s=30)
    axes[0, 0].set_xlabel(f'PC1 ({explained_var[0]*100:.1f}%)')
    axes[0, 0].set_ylabel(f'PC2 ({explained_var[1]*100:.1f}%)')
    axes[0, 0].set_title('PCA', fontweight='bold', fontsize=14)
    axes[0, 0].legend()
    
    # t-SNE (perplexity=30)
    tsne = TSNE(n_components=2, perplexity=30, random_state=42, max_iter=1000)
    X_tsne = tsne.fit_transform(X_pca[:, :n_components_tsne])
    for substance in SUBSTANCES:
        mask = substances == substance
        axes[0, 1].scatter(X_tsne[mask, 0], X_tsne[mask, 1], c=colors[substance], 
                          label=substance, alpha=0.6, s=30)
    axes[0, 1].set_xlabel('t-SNE 1')
    axes[0, 1].set_ylabel('t-SNE 2')
    axes[0, 1].set_title('t-SNE (perplexity=30)', fontweight='bold', fontsize=14)
    axes[0, 1].legend()
    
    # UMAP
    if UMAP_AVAILABLE:
        reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
        X_umap = reducer.fit_transform(X_scaled)
        for substance in SUBSTANCES:
            mask = substances == substance
            axes[1, 0].scatter(X_umap[mask, 0], X_umap[mask, 1], c=colors[substance], 
                              label=substance, alpha=0.6, s=30)
        axes[1, 0].set_xlabel('UMAP 1')
        axes[1, 0].set_ylabel('UMAP 2')
        axes[1, 0].set_title('UMAP (n_neighbors=15)', fontweight='bold', fontsize=14)
        axes[1, 0].legend()
    else:
        axes[1, 0].text(0.5, 0.5, 'UMAP not available', ha='center', va='center', fontsize=14)
        axes[1, 0].set_title('UMAP', fontweight='bold', fontsize=14)
    
    # Class distribution
    class_counts = df_analysis['substance'].value_counts()
    axes[1, 1].bar(class_counts.index, class_counts.values, 
                   color=[colors[s] for s in class_counts.index], edgecolor='black')
    axes[1, 1].set_xlabel('Substance')
    axes[1, 1].set_ylabel('Sample Count')
    axes[1, 1].set_title('Class Distribution', fontweight='bold', fontsize=14)
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    # Add metrics text
    metrics_text = f"Silhouette: {separability_metrics['Silhouette_Score']:.3f}\n"
    metrics_text += f"Calinski-Harabasz: {separability_metrics['Calinski_Harabasz_Index']:.1f}\n"
    metrics_text += f"Davies-Bouldin: {separability_metrics['Davies_Bouldin_Index']:.3f}"
    axes[1, 1].text(0.02, 0.98, metrics_text, transform=axes[1, 1].transAxes, 
                    fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))
    
    plt.suptitle('Class Separability Analysis', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(data_path / '12_combined_separability.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("✓ Saved: 12_combined_separability.png")
    
    # --- VERIFICATION ---
    print("\n>>> PRINT VERIFICATION:")
    print(f"    ✓ PCA analysis saved: 09_pca_analysis.png")
    print(f"    ✓ t-SNE analysis saved: 10_tsne_analysis.png")
    if UMAP_AVAILABLE:
        print(f"    ✓ UMAP analysis saved: 11_umap_analysis.png")
    print(f"    ✓ Combined visualization saved: 12_combined_separability.png")
    print(f"    ✓ Silhouette Score: {separability_metrics['Silhouette_Score']:.4f}")
    

    print("STEP 5: Feature Engineering")
    # Work with full dataset
    df_eng = df.copy()
    
    # 5.1 Gas Resistance Features
    print("\n 5.1 Gas Resistance Features ")
    
    gas_res_cols = [c for c in numeric_cols if 'gas_res' in c]
    if gas_res_cols:
        # Mean, std, min, max across sensors
        df_eng['gas_res_mean'] = df_eng[gas_res_cols].mean(axis=1)
        df_eng['gas_res_std'] = df_eng[gas_res_cols].std(axis=1)
        df_eng['gas_res_min'] = df_eng[gas_res_cols].min(axis=1)
        df_eng['gas_res_max'] = df_eng[gas_res_cols].max(axis=1)
        df_eng['gas_res_range'] = df_eng['gas_res_max'] - df_eng['gas_res_min']
        df_eng['gas_res_cv'] = df_eng['gas_res_std'] / df_eng['gas_res_mean']  # Coefficient of variation
        
        # Ratios between sensors
        if len(gas_res_cols) >= 2:
            df_eng['gas_res_ratio_1_8'] = df_eng[gas_res_cols[0]] / df_eng[gas_res_cols[-1]].replace(0, np.nan)
            df_eng['gas_res_ratio_max_min'] = df_eng['gas_res_max'] / df_eng['gas_res_min'].replace(0, np.nan)
        
        print(f"  Created {8} gas resistance aggregate features")
    
    # 5.2 Temperature & Humidity Features
    print("\n 5.2 Temperature & Humidity Features ")
    
    temp_cols = [c for c in numeric_cols if 'temp' in c]
    hum_cols = [c for c in numeric_cols if 'hum' in c]
    
    if temp_cols:
        df_eng['temp_mean'] = df_eng[temp_cols].mean(axis=1)
        print(f"  Created temp_mean feature")
    
    if hum_cols:
        df_eng['hum_mean'] = df_eng[hum_cols].mean(axis=1)
        print(f"  Created hum_mean feature")
    
    # 5.3 Sensor Profile Features
    print("\n 5.3 Sensor Profile Features ")
    
    # Slope across gas resistance sensors (indicates sensor response pattern)
    if len(gas_res_cols) >= 3:
        # Calculate slope using linear regression across sensors
        sensor_indices = np.arange(len(gas_res_cols))
        
        def calc_slope(row):
            values = row[gas_res_cols].values.astype(float)
            valid = ~pd.isna(values)
            if valid.sum() >= 2:
                try:
                    return np.polyfit(sensor_indices[valid], values[valid], 1)[0]
                except:
                    return np.nan
            return np.nan
        
        df_eng['gas_res_slope'] = df_eng.apply(calc_slope, axis=1)
        print(f"  Created gas_res_slope feature")
    
    # 5.4 Save Engineered Features
    engineered_cols = ['gas_res_mean', 'gas_res_std', 'gas_res_min', 'gas_res_max', 
                       'gas_res_range', 'gas_res_cv', 'gas_res_ratio_1_8', 
                       'gas_res_ratio_max_min', 'temp_mean', 'hum_mean', 'gas_res_slope']
    engineered_cols = [c for c in engineered_cols if c in df_eng.columns]
    
    print(f"\n  Total engineered features: {len(engineered_cols)}")

    
    print("\n>>> PRINT VERIFICATION:")
    print(f"    ✓ Engineered features: {engineered_cols}")
    
   

    print("STEP 6: Prepare ML-Ready Data Structures")
    # Filter to substance measurements only
    df_ml = df_eng[df_eng['measurement_type'] != 'air'].copy()
    
    # Define feature sets
    all_features = numeric_cols + engineered_cols
    all_features = [c for c in all_features if c in df_ml.columns]
    
    # Remove features with too many missing values
    valid_features = []
    for col in all_features:
        missing_pct = df_ml[col].isnull().sum() / len(df_ml) * 100
        if missing_pct < 30:
            valid_features.append(col)
    
    print(f"\n  Total features: {len(all_features)}")
    print(f"  Valid features (< 30% missing): {len(valid_features)}")
    
    # Prepare X and y
    X = df_ml[valid_features].copy()
    y_substance = df_ml['substance'].copy()
    
    # Impute missing values
    imputer = SimpleImputer(strategy='median')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=valid_features, index=X.index)
    
    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y_substance)
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=valid_features, index=X.index)
    
    # 6.1 Train/Validation/Test Split
    print("\n 6.1 Train/Validation/Test Split ")
    
    # First split: 80% train+val, 20% test
    X_trainval, X_test, y_trainval, y_test = train_test_split(
        X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    # Second split: 75% train, 25% val (of the 80%)
    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval, test_size=0.25, random_state=42, stratify=y_trainval
    )
    
    print(f"  Training set: {len(X_train)} samples ({len(X_train)/len(X_scaled)*100:.1f}%)")
    print(f"  Validation set: {len(X_val)} samples ({len(X_val)/len(X_scaled)*100:.1f}%)")
    print(f"  Test set: {len(X_test)} samples ({len(X_test)/len(X_scaled)*100:.1f}%)")
    
    # 6.2 Save ML-Ready Datasets
    print("\n 6.2 Saving ML-Ready Datasets ")
    
    # Training set
    train_df = X_train.copy()
    train_df['target'] = y_train
    train_df['substance'] = le.inverse_transform(y_train)
    train_df.to_csv(data_path / 'ml_train_final.csv', index=False)
    print(f"  ✓ Saved: ml_train_final.csv")
    
    # Validation set
    val_df = X_val.copy()
    val_df['target'] = y_val
    val_df['substance'] = le.inverse_transform(y_val)
    val_df.to_csv(data_path / 'ml_val_final.csv', index=False)
    print(f"  ✓ Saved: ml_val_final.csv")
    
    # Test set
    test_df = X_test.copy()
    test_df['target'] = y_test
    test_df['substance'] = le.inverse_transform(y_test)
    test_df.to_csv(data_path / 'ml_test_final.csv', index=False)
    print(f"  ✓ Saved: ml_test_final.csv")
    
    # Full scaled dataset
    full_df = X_scaled.copy()
    full_df['target'] = y_encoded
    full_df['substance'] = y_substance.values
    full_df.to_csv(data_path / 'ml_full_scaled.csv', index=False)
    print(f"  ✓ Saved: ml_full_scaled.csv")
    
    # Unscaled (for tree-based models)
    unscaled_df = X_imputed.copy()
    unscaled_df['target'] = y_encoded
    unscaled_df['substance'] = y_substance.values
    unscaled_df.to_csv(data_path / 'ml_full_unscaled.csv', index=False)
    print(f"  ✓ Saved: ml_full_unscaled.csv")
    
    # 6.3 Save Metadata
    print("\n 6.3 Saving Metadata ")
    
    metadata = {
        'feature_names': valid_features,
        'n_features': len(valid_features),
        'n_samples_total': len(X_scaled),
        'n_train': len(X_train),
        'n_val': len(X_val),
        'n_test': len(X_test),
        'class_labels': {name: int(idx) for idx, name in enumerate(le.classes_)},
        'class_distribution': {
            'train': {le.classes_[i]: int((y_train == i).sum()) for i in range(len(le.classes_))},
            'val': {le.classes_[i]: int((y_val == i).sum()) for i in range(len(le.classes_))},
            'test': {le.classes_[i]: int((y_test == i).sum()) for i in range(len(le.classes_))}
        },
        'engineered_features': engineered_cols,
        'original_features': [c for c in valid_features if c not in engineered_cols],
        'separability_metrics': {k: float(v) for k, v in separability_metrics.items()}
    }
    
    with open(data_path / 'ml_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"  ✓ Saved: ml_metadata.json")
    
    # Save scaler and encoder
    import pickle
    with open(data_path / 'scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)
    with open(data_path / 'label_encoder.pkl', 'wb') as f:
        pickle.dump(le, f)
    with open(data_path / 'imputer.pkl', 'wb') as f:
        pickle.dump(imputer, f)
    print(f"  ✓ Saved: scaler.pkl, label_encoder.pkl, imputer.pkl")
    
    # 6.4 Create K-Fold indices
    print("\n 6.4 Creating K-Fold Cross-Validation Indices ")
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_indices = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y_encoded)):
        fold_indices.append({
            'fold': fold + 1,
            'train_indices': train_idx.tolist(),
            'val_indices': val_idx.tolist()
        })
    
    with open(data_path / 'kfold_indices.json', 'w') as f:
        json.dump(fold_indices, f)
    print(f"  ✓ Saved: kfold_indices.json (5-fold CV)")
    
    # --- VERIFICATION ---
    print("\n>>> SECTION 6 VERIFICATION:")
    print(f"    ✓ Training samples: {len(X_train)}")
    print(f"    ✓ Validation samples: {len(X_val)}")
    print(f"    ✓ Test samples: {len(X_test)}")
    print(f"    ✓ Features: {len(valid_features)}")
    print(f"    ✓ Classes: {list(le.classes_)}")
    

    print("SECTION 7: Data Quality Report")
    quality_report = {
        'dataset_info': {
            'total_samples': len(df),
            'substance_samples': len(df_ml),
            'air_samples': len(df) - len(df_ml),
            'features_original': len(numeric_cols),
            'features_engineered': len(engineered_cols),
            'features_final': len(valid_features)
        },
        'missing_data': {
            'total_missing_pct': (df[numeric_cols].isnull().sum().sum() / (len(df) * len(numeric_cols)) * 100),
            'features_dropped': len(all_features) - len(valid_features)
        },
        'class_balance': {
            'min_class_size': int(df_ml['substance'].value_counts().min()),
            'max_class_size': int(df_ml['substance'].value_counts().max()),
            'imbalance_ratio': df_ml['substance'].value_counts().max() / df_ml['substance'].value_counts().min()
        },
        'separability': separability_metrics,
        'recommendations': []
    }
    
    with open(data_path / 'data_quality_report.json', 'w') as f:
        json.dump(quality_report, f, indent=2)
    
    print("\nData Quality Summary:")
    print(f"  Total Samples: {quality_report['dataset_info']['total_samples']}")
    print(f"  Final Features: {quality_report['dataset_info']['features_final']}")
    print(f"  Missing Data: {quality_report['missing_data']['total_missing_pct']:.2f}%")
    print(f"  Class Imbalance Ratio: {quality_report['class_balance']['imbalance_ratio']:.2f}")
    print(f"  Silhouette Score: {separability_metrics['Silhouette_Score']:.4f}")
    
    print("\nRecommendations:")
    for rec in quality_report['recommendations']:
        print(f"  • {rec}")
    
    # --- VERIFICATION ---
    print("\n>>> SECTION 7 VERIFICATION:")
    print(f"    ✓ Data quality report saved: data_quality_report.json")
    
    # ==========================================================
    # FINAL SUMMARY
    # ==========================================================
    print("\n" + "=" * 70)
    print("STEP-3 FINAL SUMMARY")
    print("=" * 70)
    
    print(f"\n📁 All outputs saved to: {data_path.absolute()}")
    
    print("\n📊 STATISTICAL ANALYSIS:")
    print("   • detailed_substance_statistics.csv")
    print("   • detailed_feature_statistics.csv")
    print("   • anova_results.csv")
    
    print("\n📈 FEATURE IMPORTANCE:")
    print("   • mutual_information_scores.csv")
    print("   • f_scores.csv")
    print("   • combined_feature_ranking.csv")
    
    print("\n🖼️  VISUALIZATIONS:")
    print("   • 09_pca_analysis.png")
    print("   • 10_tsne_analysis.png")
    if UMAP_AVAILABLE:
        print("   • 11_umap_analysis.png")
    print("   • 12_combined_separability.png")
    
    print("\n🤖 ML-READY DATA:")
    print("   • ml_train_final.csv (60% of data)")
    print("   • ml_val_final.csv (20% of data)")
    print("   • ml_test_final.csv (20% of data)")
    print("   • ml_full_scaled.csv (standardized)")
    print("   • ml_full_unscaled.csv (for tree models)")
    print("   • ml_metadata.json")
    print("   • kfold_indices.json")
    print("   • scaler.pkl, label_encoder.pkl, imputer.pkl")
    
    print("\n📋 QUALITY REPORT:")
    print("   • data_quality_report.json")
    
    print("\n" + "=" * 70)
    print(">>> FINAL VERIFICATION: ALL SECTIONS PASSED ✓")
    print("=" * 70)
    print("STEP-3 COMPLETE - Data is ready for ML model training! 🚀")
    
    return df_ml


if __name__ == "__main__":
    result = main()

DATA ANALYSIS & ML PREPARATION
Step 1: Loading Data

✓ Loaded 8949 rows × 41 columns
✓ Metadata columns: 9
✓ Numeric feature columns: 32

>>> STEP 1 VERIFICATION:
    ✓ Data shape: (8949, 41)
    ✓ Substances: ['Acetone', 'Redidlo', 'Vinegar', 'Softasept', 'Savo']
    ✓ Measurement types: ['acetone', 'air', 'redidlo', 'vinegar', 'softasept', 'savo']
Step 2: Detailed Statistical Analysis

 2.1 Basic Statistics 
  Total Samples: 8949
  Total Features: 32
  Substances: 5
  Missing Values (%): 10.89

 2.2 Per-Substance Statistics 
Substance  Total_Samples  Air_Samples  Substance_Samples  Air_Ratio  Mean_Gas_Res  Std_Gas_Res  CV_Gas_Res
  Acetone           2203         1047               1156  47.526101  39968.597133 42840.349333  107.185021
  Redidlo           2071         1070               1001  51.665862  44096.985010 44936.049527  101.902771
  Vinegar           1679          860                819  51.220965  47950.617452 50363.474700  105.031963
Softasept           1464          727  